In [18]:
#importamos las librerias necesarias para trabajar:
import pandas as pd
from bs4 import BeautifulSoup as bs
from peticiones import obtenerPrecioM2PorDistritoMadrid

todas_zonas_madrid= obtenerPrecioM2PorDistritoMadrid('todas-las-zonas')
soup_html_madrid_todas=bs(todas_zonas_madrid.text, 'lxml')
tabla_distritos = soup_html_madrid_todas.find('table', {'class': 'table-price'})
lista_de_distritos=[]

#Itero sobre todas las etiquetas tr dentro de la tabla y extraemos nombre y enlace a info de distrito"""
for tr in tabla_distritos.find_all('tr'):
    td = tr.find('td')
    if td is not None:
        a = td.find('a')
        if a is not None:
            href = a['href'].replace("/indice-precio-vivienda/madrid-capital/","")
            nombre_distrito = a.text
            lista_de_distritos.append({ "distrito": nombre_distrito, "url":href})
            
print(lista_de_distritos)

[{'distrito': 'Centro', 'url': 'centro'}, {'distrito': 'Tetuán', 'url': 'tetuan'}, {'distrito': 'Barrio de Salamanca', 'url': 'barrio-de-salamanca'}, {'distrito': 'Carabanchel', 'url': 'carabanchel'}, {'distrito': 'Chamberí', 'url': 'chamberi'}, {'distrito': 'Chamartín', 'url': 'chamartin'}, {'distrito': 'Hortaleza', 'url': 'hortaleza'}, {'distrito': 'Latina', 'url': 'latina'}, {'distrito': 'Ciudad Lineal', 'url': 'ciudad-lineal'}, {'distrito': 'Arganzuela', 'url': 'arganzuela'}, {'distrito': 'Puente de Vallecas', 'url': 'puente-de-vallecas'}, {'distrito': 'Moncloa - Aravaca', 'url': 'moncloa-aravaca'}, {'distrito': 'Retiro', 'url': 'retiro'}, {'distrito': 'San Blas', 'url': 'san-blas'}, {'distrito': 'Villaverde', 'url': 'villaverde'}, {'distrito': 'Fuencarral - El Pardo', 'url': 'fuencarral-el-pardo'}, {'distrito': 'Usera', 'url': 'usera'}, {'distrito': 'Villa de Vallecas', 'url': 'villa-de-vallecas'}, {'distrito': 'Vicálvaro', 'url': 'vicalvaro'}, {'distrito': 'Barajas', 'url': 'bara

In [19]:
#Una vez extraidas todos los distritos y sus urls, vamos a extraer el precio medio del m2 y el el precio medio por m2 de viviendas de una sola habitacion o estudios 

for distrito in lista_de_distritos:
    html_fotocasa_distrito= obtenerPrecioM2PorDistritoMadrid( distrito['url'])
    soup_fotocasa_distrito=bs(html_fotocasa_distrito.text, 'lxml')

    # Obtengo precio medio m2 por distrito
    html_m2 = soup_fotocasa_distrito.find('div', {'class': 'b-detail_title'})
   
    if html_m2 == None:
        html_m2 = soup_fotocasa_distrito.find('div', {'class': 'text-big'})
        precio_m2 = int(html_m2.text.strip().replace("€", "").replace(".", ""))
        distrito['m2_medio']=precio_m2

    else:
        precio_medio_m2 = int(html_m2.text.strip().replace("€/m²", "").replace(".", ""))
        distrito['m2_medio'] = precio_medio_m2

    # Obtengo precio medio m2 por distrito de pisos de una habitación o estudios
    html_m2_por_1hab= soup_fotocasa_distrito.find("div", class_="description-item_semibold")

    if html_m2_por_1hab == None:
         distrito['m2_estudio_q1hab']= 0
    else:
         valor_m2_por_1hab_estudio= int(html_m2_por_1hab.text.strip().replace("€/m²", "").replace(".", ""))
         distrito['m2_estudio_q1hab']=valor_m2_por_1hab_estudio
         
print(lista_de_distritos)

[{'distrito': 'Centro', 'url': 'centro', 'm2_medio': 5941, 'm2_estudio_q1hab': 5802}, {'distrito': 'Tetuán', 'url': 'tetuan', 'm2_medio': 4443, 'm2_estudio_q1hab': 4675}, {'distrito': 'Barrio de Salamanca', 'url': 'barrio-de-salamanca', 'm2_medio': 7157, 'm2_estudio_q1hab': 7504}, {'distrito': 'Carabanchel', 'url': 'carabanchel', 'm2_medio': 2501, 'm2_estudio_q1hab': 3100}, {'distrito': 'Chamberí', 'url': 'chamberi', 'm2_medio': 6228, 'm2_estudio_q1hab': 6471}, {'distrito': 'Chamartín', 'url': 'chamartin', 'm2_medio': 5590, 'm2_estudio_q1hab': 5671}, {'distrito': 'Hortaleza', 'url': 'hortaleza', 'm2_medio': 4233, 'm2_estudio_q1hab': 4592}, {'distrito': 'Latina', 'url': 'latina', 'm2_medio': 2654, 'm2_estudio_q1hab': 2927}, {'distrito': 'Ciudad Lineal', 'url': 'ciudad-lineal', 'm2_medio': 3456, 'm2_estudio_q1hab': 4024}, {'distrito': 'Arganzuela', 'url': 'arganzuela', 'm2_medio': 4362, 'm2_estudio_q1hab': 4601}, {'distrito': 'Puente de Vallecas', 'url': 'puente-de-vallecas', 'm2_medio':

In [20]:
#vamos a transformar en data frame el diccionario obtenido mas arriba:
df_precio_m2_vivienda_por_distrito = pd.DataFrame(lista_de_distritos, columns=["distrito", "url", "m2_medio", "m2_estudio_q1hab"])
df_precio_m2_vivienda_por_distrito

,distrito,url,m2_medio,m2_estudio_q1hab
0,Centro,centro,5941,5802
1,Tetuán,tetuan,4443,4675
2,Barrio de Salamanca,barrio-de-salamanca,7157,7504
3,Carabanchel,carabanchel,2501,3100
4,Chamberí,chamberi,6228,6471
5,Chamartín,chamartin,5590,5671
6,Hortaleza,hortaleza,4233,4592
7,Latina,latina,2654,2927
8,Ciudad Lineal,ciudad-lineal,3456,4024
9,Arganzuela,arganzuela,4362,4601


In [21]:
df_precio_m2_vivienda_por_distrito.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   distrito          20 non-null     object
 1   url               20 non-null     object
 2   m2_medio          20 non-null     int64 
 3   m2_estudio_q1hab  20 non-null     int64 
dtypes: int64(2), object(2)
memory usage: 768.0+ bytes


In [22]:
# Una vez detectado que tenemos una fila con un valor no encontrado = 0 tratamos de calcular con la media un valor aproximado
precio_medio_m2_vicalvaro = df_precio_m2_vivienda_por_distrito.loc[df_precio_m2_vivienda_por_distrito['distrito'] == 'Vicálvaro', 'm2_medio'].iloc[0]
precio_medio_m2_superior= df_precio_m2_vivienda_por_distrito[df_precio_m2_vivienda_por_distrito['m2_medio'] > precio_medio_m2_vicalvaro].nsmallest(1, 'm2_medio')['m2_estudio_q1hab'].iloc[0]
precio_medio_inferior= df_precio_m2_vivienda_por_distrito[df_precio_m2_vivienda_por_distrito['m2_medio'] < precio_medio_m2_vicalvaro].nlargest(1, 'm2_medio')['m2_estudio_q1hab'].iloc[0]
precio_medio_1hab_vicalvaro= int((precio_medio_m2_superior + precio_medio_inferior) / 2)


df_precio_m2_vivienda_por_distrito.loc[df_precio_m2_vivienda_por_distrito['distrito'] == 'Vicálvaro', 'm2_estudio_q1hab'] = precio_medio_1hab_vicalvaro
df_precio_m2_vivienda_por_distrito.to_csv('../data/precio_vivienda_m2_por_distrito.csv', index=False)
df_precio_m2_vivienda_por_distrito


,distrito,url,m2_medio,m2_estudio_q1hab
0,Centro,centro,5941,5802
1,Tetuán,tetuan,4443,4675
2,Barrio de Salamanca,barrio-de-salamanca,7157,7504
3,Carabanchel,carabanchel,2501,3100
4,Chamberí,chamberi,6228,6471
5,Chamartín,chamartin,5590,5671
6,Hortaleza,hortaleza,4233,4592
7,Latina,latina,2654,2927
8,Ciudad Lineal,ciudad-lineal,3456,4024
9,Arganzuela,arganzuela,4362,4601


In [16]:

#vamos a realizar una grafica mas para poder observar cuales serian los distritos mas caros en madrid:
#calculamos primero la media de precios del m2 de un estudio
media_m2_estudio_q1hab = df_precio_m2_vivienda_por_distrito['m2_estudio_q1hab'].mean()

print('La media de m2_estudio_q1hab es:', media_m2_estudio_q1hab)

La media de m2_estudio_q1hab es: 4449.35
